In [36]:
#!pip install simpletransformers

In [37]:
import pandas as pd


In [38]:
train_data = pd.read_csv('./data/olid-train.csv')
test_data = pd.read_csv('./data/olid-test.csv')
diagnostic_test_data = pd.read_csv('./data/olid-subset-diagnostic-tests.csv')

print(train_data.head())
print(test_data.head())
print(diagnostic_test_data.head())

      id                                               text  labels
0  86426  @USER She should ask a few native Americans wh...       1
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       1
2  16820  Amazon is investigating Chinese employees who ...       0
3  62688  @USER Someone should'veTaken" this piece of sh...       1
4  43605  @USER @USER Obama wanted liberals &amp; illega...       0
      id                                               text  labels
0  15923  #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...       1
1  27014  #ConstitutionDay is revered by Conservatives, ...       0
2  30530  #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...       0
3  13876  #Watching #Boomer getting the news that she is...       0
4  60133  #NoPasaran: Unity demo to oppose the far-right...       1
      id                                               text  labels
0  89200      @USER @USER Who the hell does he think he is?       1
1  71294  #BREAKING. #Greece: Molotov cocktails 

In [39]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score

In [40]:
# from https://simpletransformers.ai/docs/usage/ and https://simpletransformers.ai/docs/binary-classification/
model_args = ClassificationArgs()
model_args.num_train_epochs = 2
model_args.learning_rate = 1e-4
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/trained_models'


model = ClassificationModel("bert", '/content/drive/MyDrive/NLP/trained_models', args=model_args)


In [41]:
model

In [42]:
# Train the model
model.train_model(train_data)

  0%|          | 0/13240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1655 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1655 [00:00<?, ?it/s]

(3310, 0.6459590070557378)

In [43]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_data, report = classification_report)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
print(result['report'])

              precision    recall  f1-score   support

         0.0       0.72      1.00      0.84       620
         1.0       0.00      0.00      0.00       240

    accuracy                           0.72       860
   macro avg       0.36      0.50      0.42       860
weighted avg       0.52      0.72      0.60       860



In [45]:
print(model_outputs)

[[ 0.73242188 -0.1418457 ]
 [ 0.73242188 -0.1418457 ]
 [ 0.73242188 -0.1418457 ]
 ...
 [ 0.73242188 -0.1418457 ]
 [ 0.73242188 -0.1418457 ]
 [ 0.73242188 -0.1418457 ]]


In [46]:
test_data_list = []
for text in test_data['text']:
  test_data_list.append(text)

predictions, raw_outputs = model.predict(test_data_list)

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [47]:
predicted_labels = pd.DataFrame({'id': test_data.id,'text':test_data.text, 'predicted': predictions})
predicted_labels.head()

,id,text,predicted
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,0


In [48]:
predicted_labels.to_csv('predicted_labels.csv', index=False)

In [49]:
print("Confusion Matrix : ")
print(confusion_matrix(test_data['labels'], predictions))

Confusion Matrix : 
[[620   0]
 [240   0]]
